Importing libraries and loading environment variables from a .env file containing the API Key 

In [15]:
import requests
import tmdbsimple as tmdb
import pandas as pd
from dotenv import load_dotenv
import os

# Loading environment variables and API key
load_dotenv()
API_KEY = os.getenv("TMDb_API_KEY")
if not API_KEY:
    raise ValueError("TMDB API Key not found. Make sure it's set in the .env file.")

tmdb.API_KEY = API_KEY
base_URL = "https://api.themoviedb.org/3"
url = f"{base_URL}/person/{6384}/movie_credits?api_key={API_KEY}"

print(f"Requesting URL: {url}")
response = requests.get(url)

Requesting URL: https://api.themoviedb.org/3/person/6384/movie_credits?api_key=93b29bf0b851209878217fccd1d8ca1a


In [16]:
if response.status_code == 200: 
    try:
        # Parsing JSON response 
        movie_data = response.json()
        print(f"Request Successful!")
        
        # Extracting 'cast' information
        movies = movie_data.get('cast', [])
        
        # Creating a DataFrame with selected columns
        columns = ['title', 'release_date', 'character', 'revenue']
        refined_movies = [
            {
                'title': movie.get('title'),
                'release_date': movie.get('release_date'),
                'character': movie.get('character'),
                'revenue': movie.get('revenue')
            }
        for movie in movies
        ]
        tmdb_df = pd.DataFrame(refined_movies, columns=columns)
        print("TMDb DataFrame:")
        print(tmdb_df.head())
        
        # Loading the "Whoa API" data
        whoa_df = pd.read_csv("keanu_movies_filtered.csv")
        print("Whoa DataFrame:")
        print(whoa_df.head())
        
        # Merging the two DataFrames
        combined_df = pd.merge(
            whoa_df,
            tmdb_df,
            left_on = ['movie'],
            right_on = ['title'],
            how = 'left'
        )
        
        # Saving combined DataFrame to CSV
        combined_df.to_csv("keanu_combined_data.csv", index=False)
        print("Combined data saved to keanu_combined_data.csv")
        
    except requests.exceptions.JSONDecodeError:
        print("Error: Response could not be decoded as JSON")
else:
    print(f"Error: {response.status_code} - {response.reason}")

Request Successful!
TMDb DataFrame:
                  title release_date                     character revenue
0    Dangerous Liaisons   1988-12-21  Le Chevalier Raphael Danceny    None
1  My Own Private Idaho   1991-02-01                   Scott Favor    None
2           Point Break   1991-07-12                   Johnny Utah    None
3           Constantine   2005-02-08              John Constantine    None
4          Street Kings   2008-04-10          Detective Tom Ludlow    None


FileNotFoundError: [Errno 2] No such file or directory: 'keanu_movies_filtered.csv'

In [17]:
url = "https://whoa.onrender.com/whoas/ordered/1-118"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    unique_movies = {movie['movie']: movie for movie in data}.values()
    df = pd.DataFrame(unique_movies)
    
    # Selected columns
    columns_to_keep = ['movie', 'year', 'release_date', 'character', 'total_whoas_in_movie']
    filtered_df = df[columns_to_keep]
    
    # Exporting to CSV
    filtered_df.to_csv("keanu_movies_filtered.csv", index=False)
    print("Filtered data saved to keanu_movies.csv")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

Filtered data saved to keanu_movies.csv


This code block is specifically for fetching the revenue data

In [22]:
merged_file = 'keanu_movies_filtered.csv'
merged_data = pd.read_csv(merged_file)

revenues = []
for index, row in merged_data.iterrows():
    movie_title = row['title']
    release_date = ['release_date_y']
    movie_id = row.get('movie_id')
    
    if pd.isna(movie_title):
        revenues.append(None)
        continue
    
    # Fetch movie details using movie_id
    details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}"
    resonse = requests.get(details_url)

    if response.status_code == 200:
        movie_details = response.json()
        revenue = movie_details.get('revenue', None)
        revenues.append(revenue)
    else:
        print(f"Failed to fetch revenue for {movie_title}")
        revenues.append(None)

merged_data['revenue'] = revenues
merged_data.to_csv("keanu_movies_with_revenue.csv", index=False)
print("Updated dataset saved to keanu_movies_with_revenue.csv")

KeyError: 'title'